# Analysis of A05 results

## Data loading & filtering

In [ ]:
from curve_analyzer.utils.data_processing import load_curves, load_trait
from curve_analyzer.visual.widgets import get_filtering_widgets, filter_df_via_widgets

trait_name = "a05"
curves = load_curves()
df_trait = load_trait(trait_name) # load all results for the given trait
df_trait = df_trait.merge(curves, "left", "curve") # join curve metadata to trait results
filtering_widgets = get_filtering_widgets(trait_name)

## Data inspection

In [ ]:
df = filter_df_via_widgets(df_trait, trait_name, filtering_widgets)
df

In [ ]:
df.agg(set) # display only distinct values

In [ ]:
df.describe()

## Data visualization

In [ ]:
from curve_analyzer.visual.visualization import normalized_barplot, normalized_bubbleplot

In [ ]:
for column in ("least", "full"):
    normalized_barplot(df_a05, column, title=f"Normalized barplot of {column}")

In [ ]:
for l in [2,3]:
    normalized_bubbleplot(df[df["l"] == l], f"least", f"full", title=f"Normalized bubble plot for l={l}")

## Playground